# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [22]:
# Write your code below.

from dotenv import load_dotenv
import os
from glob import glob
load_dotenv()
price_data_path = os.getenv("PRICE_DATA")
print("PRICE_DATA =", price_data_path)
files = glob(os.path.join(price_data_path, "*.parquet"))
files[:3]


PRICE_DATA = ../../05_src/data/prices/


[]

In [23]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [24]:
import os
from glob import glob

# Get the path from the environment variable
price_data_path = os.getenv("PRICE_DATA")

# 🔁 Search all subfolders for .parquet files
parquet_files = glob(os.path.join(price_data_path, "**/*.parquet"), recursive=True)

print("Parquet files found:", parquet_files[:5])


Parquet files found: ['../../05_src/data/prices\\ABBV\\ABBV_2013\\part.0.parquet', '../../05_src/data/prices\\ABBV\\ABBV_2013\\part.1.parquet', '../../05_src/data/prices\\ABBV\\ABBV_2014\\part.0.parquet', '../../05_src/data/prices\\ABBV\\ABBV_2014\\part.1.parquet', '../../05_src/data/prices\\ABBV\\ABBV_2015\\part.0.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.

import dask.dataframe as dd

# Read the data
ddf = dd.read_parquet(parquet_files)

# Check available columns
print(ddf.columns)  

ddf = ddf.assign(
    Close_lag_1 = ddf.Close.shift(1)
)

ddf = ddf.assign(
    Adj_Close_lag_1 = ddf["Adj Close"].shift(1)  
)

# Add calculated features
ddf = ddf.assign(
    returns = (ddf.Close / ddf.Close_lag_1) - 1,
    hi_lo_range = ddf.High - ddf.Low
)

dd_feat = ddf


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'ticker', 'Year'],
      dtype='object')


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [26]:
# Write your code below.

df = dd_feat.compute()  # Converts the Dask DataFrame to a pandas DataFrame by executing all lazy operations. Use only when data fits into memory.


df["returns_ma10"] = df["returns"].rolling(10).mean()  # Calculates the 10-day moving average of the returns column. Useful for trend smoothing.

df[["Date", "returns", "returns_ma10"]].head(15)

,Date,returns,returns_ma10
0,2013-01-02,NaN,NaN
1,2013-01-03,-0.008257,NaN
2,2013-01-04,-0.012633,NaN
3,2013-01-07,0.002035,NaN
4,2013-01-08,-0.021764,NaN
5,2013-01-09,0.005636,NaN
6,2013-01-10,0.002950,NaN
7,2013-01-11,-0.004412,NaN
8,2013-01-14,0.007090,NaN
9,2013-01-15,0.014960,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?


Answer- 
Yes, for simplicity. Pandas provides an easy-to-use .rolling().mean() method that works well for calculating moving averages. Dask supports it too, but it’s more limited and requires additional parameters for accurate results.


+ Would it have been better to do it in Dask? Why?

Answer- 

If the dataset is very large and cannot fit into memory, then doing it in Dask would be better. Dask processes data in chunks and avoids memory crashes. But calculating a moving average in Dask needs more setup using map_overlap() to handle overlapping windows across partitions. So unless the data is huge, pandas is faster and simpler.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.